In [1]:
from os import listdir
from os.path import isfile, join
import random
import numpy as np
import string
from huffman import HuffmanCoding
from huffmancodec import HuffmanCodec
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
# obtengo los nombres de los archivos en una carpeta
folder = 'sci.med'
files = [f for f in listdir('./20news-bydate-train/' + folder) if isfile(join('./20news-bydate-train/' + folder, f))]

# obtengo una cantidad n de archivos aleatorios
numero_archivos = 500
random_files = random.sample(files, numero_archivos)
text_concatenation = ''
for file in random_files:
    f=open('./20news-bydate-train/'+ folder + '/' + file, "r")
    try:
        lines = f.readlines()
    except UnicodeDecodeError:
        lines = []
    text = ''
    for line in lines:
        text += line[:-1]
    f.close()
    text_concatenation += text
print(text)

From: noring@netcom.com (Jon Noring)Subject: Good Grief!  (was Re: Candida Albicans: what is it?)Organization: Netcom Online Communications Services (408-241-9760 login: guest)Lines: 81In article rind@enterprise.bih.harvard.edu (David Rind) writes:>In article davpa@ida.liu.se  (David Partain) writes:>>Someone I know has recently been diagnosed as having Candida Albicans, >>a disease about which I can find no information.  Apparently it has something>>to do with the body's production of yeast while at the same time being highly>>allergic to yeast.  Can anyone out there tell me any more about it?>Candida albicans can cause severe life-threatening infections, usually>in people who are otherwise quite ill.  This is not, however, the sort>of illness that you are probably discussing.>>"Systemic yeast syndrome" where the body is allergic to>yeast is considered a quack diagnosis by mainstream medicine.  There>is a book "The Yeast Connection" which talks about this "illness".>>There is no convi

In [3]:
# creo una muestra aleatoria de ejemplos
numero_datos = 30000
tamaño_ventana = 5
random_indexes = random.sample(range(len(text_concatenation) - tamaño_ventana), numero_datos)
random_samples = []
for i in random_indexes:
    sample = text_concatenation[i:i+tamaño_ventana]
    random_samples.append(sample)

In [4]:
# hago un diccionario one hot con todos los caracteres ascii
chars = string.printable[:-5]
chars += '\n'
for i in range(161, 255):
    chars += chr(i)
print(chars)
one_hot = {}
for i, l in enumerate(chars):
    vector = np.zeros(190)
    vector[i] = 1
    one_hot[l] = vector
print(one_hot.keys())

def window_onehot(window, one_hot):
    inp = []
    for char in window[:-1]:
        try:
            inp.append(one_hot[char])
        except KeyError:
            if (char == '\t'):
                inp.append(one_hot['t'])
            elif (char == '\r'):
                inp.append(one_hot['r'])
            else:
                inp.append(one_hot['\\'])
    sum_vector = np.zeros(190)
    for vector in inp:
        sum_vector += vector
    return sum_vector

# para cada ejemplo de la muestras de ejemplos lo paso a one hot
input_samples = []
output_samples = []
for sample in random_samples:
    sum_vector = window_onehot(sample, one_hot)
    input_samples.append(sum_vector)
    
    try:
        output_samples.append(one_hot[sample[-1]])
    except KeyError:
        if (sample[-1] == '\t'):
            output_samples.append(one_hot['t'])
        elif (sample[-1] == '\r'):
            output_samples.append(one_hot['r'])
        else:
            output_samples.append(one_hot['\\'])
            
output_samples = []
for sample in random_samples:
    #if sample[-1] == ' ':
    #output_samples.append(random.choice(chars))
    #else:
    #   output_samples.append(sample[-1])
    output_samples.append(sample[-1])
#le = preprocessing.LabelEncoder()
#le.fit(input_samples)
#input_samples = le.transform(input_samples)
#output_samples = le.transform(output_samples)
train_inputs = input_samples[:int((len(input_samples)/3)*2)]
train_outputs = output_samples[:int((len(input_samples)/3)*2)]
test_inputs = input_samples[int((len(input_samples)/3)*2):]
test_outputs = output_samples[int((len(input_samples)/3)*2):]

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 
¡¢£¤¥¦§¨©ª«¬­®¯°±²³´µ¶·¸¹º»¼½¾¿ÀÁÂÃÄÅÆÇÈÉÊËÌÍÎÏÐÑÒÓÔÕÖ×ØÙÚÛÜÝÞßàáâãäåæçèéêëìíîïðñòóôõö÷øùúûüýþ
dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' ', '\n', '¡', '¢', '£', '¤', '¥', '¦', '§', '¨', '©', 'ª', '«', '¬', '\xad', '®', '¯', '°', '±', '²', '³', '´', 'µ', '¶', '·', '¸', '¹', 'º', '»', '¼', '½', '¾', '¿', 'À', 'Á', 'Â', 'Ã', 'Ä', 'Å', 'Æ', 'Ç', 'È', 'É', 'Ê', 'Ë', 'Ì', 'Í', 'Î', 'Ï', 'Ð', 'Ñ', 'Ò', 'Ó', 'Ô', 'Õ', 'Ö', '×', 'Ø', 'Ù', 'Ú', 'Û', 'Ü', 'Ý', 'Þ', 'ß'

In [5]:
def model_performance(classifier, test_inputs, test_outputs):
    probs = classifier.predict_proba(test_inputs)
    pred = classifier.predict(test_inputs)
    print(probs)
    print(pred)
    matrix = confusion_matrix(test_outputs, pred)
    acc = accuracy_score(test_outputs, pred)
    #print(matrix)
    print('Precisión: ' + str(acc))
    return probs

def train_model(train_inputs, train_outputs, penalization):
    classifier = RandomForestClassifier(n_estimators=100,random_state=0)
    classifier.fit(train_inputs, train_outputs)
    #classifier = SVC(decision_function_shape='ovo', C=penalization, probability=True)
    #classifier.fit(train_inputs, train_outputs)
    return classifier

In [6]:
classifier = train_model(train_inputs, train_outputs, 2)

print('Desempeño con set de test:')
probs = model_performance(classifier, test_inputs, test_outputs)
print('Desempeño con set de entrenamiento:')
model_performance(classifier, train_inputs, train_outputs)


Desempeño con set de test:
[[ 0.          0.          0.01375    ...,  0.          0.          0.        ]
 [ 0.          0.          0.89484502 ...,  0.00645166  0.          0.        ]
 [ 0.          0.          0.27921491 ...,  0.01696491  0.          0.        ]
 ..., 
 [ 0.          0.          0.11216017 ...,  0.01        0.          0.        ]
 [ 0.          0.          0.80829606 ...,  0.          0.          0.        ]
 [ 0.          0.          0.09889009 ...,  0.          0.          0.        ]]
['s' ' ' ' ' ..., 'e' ' ' 'P']
Precisión: 0.3429
Desempeño con set de entrenamiento:
[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.379221   ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.          0.89484502 ...,  0.00645166  0.          0.        ]
 [ 0.          0.          0.01285714 ...,  0.          0.          0.   

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.379221  , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.89484502, ...,  0.00645166,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.01285714, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.34180245, ...,  0.        ,
         0.        ,  0.        ]])

In [7]:
#initial_window = np.zeros(189)
#for i in range(7):
#    random_index = random.randint(0, 189)
#    initial_window[random_index] += 1
#print(initial_window)
#probs = classifier.predict_proba([initial_window])
classes = classifier.classes_
initial_window = ''
for i in range(tamaño_ventana - 1):
    initial_window += random.choice(string.printable)
print(initial_window)
initial_onehot = window_onehot(initial_window, one_hot)

probs = classifier.predict_proba([initial_onehot])
print(probs)

M_U}
[[ 0.          0.          0.20667586  0.          0.          0.04        0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.01        0.          0.          0.01        0.          0.          0.
   0.          0.          0.          0.00949153  0.0175      0.          0.
   0.02306061  0.          0.04        0.01        0.01        0.          0.0325
   0.          0.04        0.          0.          0.01        0.          0.02
   0.02333333  0.          0.          0.03666667  0.04        0.03        0.
   0.          0.          0.06        0.          0.          0.          0.
   0.          0.01        0.13677201  0.          0.01        0.          0.
   0.01        0.04        0.          0.          0.          0.05        0.
   0.          0.          0.          0.044       0.          0.          0.
   0.01        0.          0.02        0.          0.

In [12]:
def create_freq_dic():
    dic = {}
    for c in chars:
        dic[c] = 0
    return dic

def probs_to_freq(probs, dic, classifier):
    classes = classifier.classes_
    classes_list = list(classes)
    for c, value in enumerate(classes_list):
        dic[value] = int(probs[0][c]*1000)
    return dic

def get_probs(window, classifier, one_hot):
    window_one_hot = window_onehot(window, one_hot)
    probs = classifier.predict_proba([window_one_hot])
    return probs

def get_codes(freqs, file):
    #codec = HuffmanCodec.from_frequencies(freqs)
    #return codec.get_code_table()
    h = HuffmanCoding(file)
    codes, reverse_codes = h.get_codes(freqs)
    return codes

In [35]:
# obtengo los nombres de los archivos en una carpeta
folder = 'sci.med'
files = [f for f in listdir('./20news-bydate-train/' + folder) if isfile(join('./20news-bydate-train/' + folder, f))]

file = random.sample(files, 1)
file_name = './20news-bydate-train/'+ folder + '/' + file[0]
print(file_name)

#función obtenida de http://bhrigu.me/blog/2017/01/17/huffman-coding-python-implementation/
def pad_encoded_text(encoded_text):
		extra_padding = 8 - len(encoded_text) % 8
		for i in range(extra_padding):
			encoded_text += "0"

		padded_info = "{0:08b}".format(extra_padding)
		encoded_text = padded_info + encoded_text
		return encoded_text
    
#función obtenida de http://bhrigu.me/blog/2017/01/17/huffman-coding-python-implementation/
def get_byte_array(padded_encoded_text):
		if(len(padded_encoded_text) % 8 != 0):
			print("Encoded text not padded properly")
			exit(0)

		b = bytearray()
		for i in range(0, len(padded_encoded_text), 8):
			byte = padded_encoded_text[i:i+8]
			b.append(int(byte, 2))
		return b

def to_compress_file(file_name, initial_window):
    text_to_compress = initial_window
    f=open(file_name, "r")
    try:
        lines = f.readlines()
    except UnicodeDecodeError:
        lines = []
    text = ''
    for line in lines:
        text += line
    f.close()
    text_to_compress += text
    print(text_to_compress)
    return text_to_compress

def process_window(window, classifier, one_hot, char_dictionary, file):
    prefijo = window[:-1]
    charac = window[-1]
    probs = get_probs(prefijo, classifier, one_hot)
    freqs = probs_to_freq(probs, char_dictionary, classifier)
    codes = get_codes(freqs, file)
    code = codes[charac]
    return code

def compress(file, classifier, one_hot, initial_window, tamaño_ventana, output_path):
    # creo el texto que debo comprimir, con la ventana inicial
    text_to_compress = to_compress_file(file, initial_window)
    char_dictionary = create_freq_dic()
    
    #para cada ventana, generar el código que corresponde a la ventana 
    encoded_text = ''
    for i in range((len(text_to_compress) - tamaño_ventana)):
        window = text_to_compress[i:i+tamaño_ventana]
        code = process_window(window, classifier, one_hot, char_dictionary, file)
        encoded_text += code
    print(len(encoded_text))
    padded_encoded_text = pad_encoded_text(encoded_text)

    b = get_byte_array(padded_encoded_text)
    with open(output_path, 'wb') as output:
        output.write(bytes(b))
    output.close()    
        
compress(file_name, classifier, one_hot, initial_window, tamaño_ventana, 'compressed.bin')
    

./20news-bydate-train/sci.med/59051
M_U}From: nyeda@cnsvax.uwec.edu (David Nye)
Subject: Re: seizures ( infantile spasms )
Organization: University of Wisconsin Eau Claire
Lines: 19

[reply to dufault@lftfld.enet.dec.com (MD)]
 
>After many metabolic tests, body structure tests, and infection/virus
>tests the doctors still do not know quite what type of siezures he is
>having (although they do have alot of evidence that it is now pointing
>to infantile spasms ).  This is where we stand right now....As I know
>now, these particular types of disorders are still not really well
>understood by the medical community.
 
Infantile spasms have been well understood for quite some time now.  You
are seeing a pediatric neurologist, aren't you?  If not, I strongly
recommend it.  There is a new anticonvulsant about to be released called
felbamate which may be particularly helpful for infantile spasms.  As
for learning more about seizures, ask your doctor or his nurse about a
local support group.
 


In [39]:
def read_file(path):
    with open(path, 'rb') as file:
        bit_string = ""
        byte = file.read(1)
        while(len(byte) > 0):
            byte = ord(byte)
            bits = bin(byte)[2:].rjust(8, '0')
            bit_string += bits
            byte = file.read(1)
    encoded_text = remove_padding(bit_string)
    return encoded_text

            
def decode_text(encoded_text, reverse_coding):
    current_code = ""
    decoded_text = ""

    for bit in encoded_text:
        current_code += bit
        if(current_code in reverse_coding):
            character = reverse_coding[current_code]
            decoded_text += character
            current_code = ""

    return decoded_text
    
def remove_padding(padded_encoded_text):
    padded_info = padded_encoded_text[:8]
    extra_padding = int(padded_info, 2)

    padded_encoded_text = padded_encoded_text[8:] 
    encoded_text = padded_encoded_text[:-1*extra_padding]

    return encoded_text

def get_reverse_codes(freqs, file):
    #codec = HuffmanCodec.from_frequencies(freqs)
    #return codec.get_code_table()
    h = HuffmanCoding(file)
    codes, reverse_codes = h.get_codes(freqs)
    return reverse_codes

def deprocess_window(window, classifier, one_hot, char_dictionary, file, encoded_text):
    prefijo = window
    probs = get_probs(prefijo, classifier, one_hot)
    freqs = probs_to_freq(probs, char_dictionary, classifier)
    reverse_codes = get_reverse_codes(freqs, file)
    
    current_code = ""
    for bit in encoded_text:
        current_code += bit
        if(current_code in reverse_codes):
            character = reverse_codes[current_code]
            break
        else:
            character = ''

    return character, len(current_code)
        

def decompress(initial_window, path, classifier, one_hot):
    encoded_text = read_file(path)
    char_dictionary = create_freq_dic()
    
    window = initial_window
    decoded_text = ''
    while len(encoded_text) > 0:
        charac, len_code = deprocess_window(window, classifier, one_hot, char_dictionary, path, encoded_text)
        decoded_text += charac
        encoded_text = encoded_text[len_code:]
        window = window[1:] + charac
    print(decoded_text)


decompress(initial_window,'compressed.bin', classifier, one_hot)


From: nyeda@cnsvax.uwec.edu (David Nye)
Subject: Re: seizures ( infantile spasms )
Organization: University of Wisconsin Eau Claire
Lines: 19

[reply to dufault@lftfld.enet.dec.com (MD)]
 
>After many metabolic tests, body structure tests, and infection/virus
>tests the doctors still do not know quite what type of siezures he is
>having (although they do have alot of evidence that it is now pointing
>to infantile spasms ).  This is where we stand right now....As I know
>now, these particular types of disorders are still not really well
>understood by the medical community.
 
Infantile spasms have been well understood for quite some time now.  You
are seeing a pediatric neurologist, aren't you?  If not, I strongly
recommend it.  There is a new anticonvulsant about to be released called
felbamate which may be particularly helpful for infantile spasms.  As
for learning more about seizures, ask your doctor or his nurse about a
local support group.
 
David Nye (nyeda@cnsvax.uwec.edu).  Mide